# 🗣️ Humanize DB - Query your SQL Database with Natural Language

## 📋 Project Overview

**Humanize DB** is an intelligent SQL database agent that allows users to interact with SQL databases using natural human language. Instead of writing complex SQL queries, simply ask questions in plain English and get accurate, formatted answers.

This project leverages **LangGraph** to create a sophisticated multi-agent workflow that:
- Understands natural language questions
- Explores database schemas automatically
- Generates and validates SQL queries
- Executes queries safely
- Returns human-readable answers

---



## 📝 Project Structure

```
humandb.ipynb
├── Setup & Configuration
│   ├── Import dependencies
│   ├── Initialize LLM (GPT-4)
│   └── Download sample database
├── Database Tools
│   ├── SQL query executor
│   ├── Schema inspector
│   └── Table lister
├── Agent Nodes
│   ├── list_tables() - Discover tables
│   ├── call_get_schema() - Get schema
│   ├── generate_query() - Create SQL
│   ├── check_query() - Validate SQL
│   └── format responses
├── Workflow Definition
│   └── LangGraph StateGraph
└── User Interfaces
    ├── Programmatic interface
    └── Gradio Chat UI
```



In [ ]:
from langchain.chat_models import init_chat_model
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from typing import Literal
from langchain_core.messages import AIMessage
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
import requests
import gradio as gr

llm = init_chat_model("openai:gpt-4.1")

## Configure the database

In [ ]:
url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"
DB_NAME = "sample.db"

response = requests.get(url)

if response.status_code == 200:
    with open(DB_NAME, "wb") as file:
        file.write(response.content)
    print(f"file downloaded and saved as {DB_NAME}")
else:
    print(f"Failed to download File. Error : {response.status_code}")

In [ ]:
db = SQLDatabase.from_uri(f"sqlite:///{DB_NAME}")

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

### Tools

I am using some of Langchain's tools specifically built for querying SQL, and displaying the schema

In [ ]:
get_schema_tool = next(tool for tool in tools if tool.name == "sql_db_schema")
run_query_tool = next(tool for tool in tools if tool.name == "sql_db_query")
list_tables_tool = next(tool for tool in tools if tool.name == "sql_db_list_tables")

In [ ]:
get_schema_node = ToolNode([get_schema_tool], name="get_schema")
run_query_node = ToolNode([run_query_tool], name="run_query")

In [ ]:
def list_tables(state: MessagesState):
    tool_call = {
        "name": "sql_db_list_tables",
        "args": {},
        "id": "abc123",
        "type": "tool_call",
    }

    tool_call_message = AIMessage(content="", tool_calls=[tool_call])
    tool_message = list_tables_tool.invoke(tool_call)
    response = AIMessage(f"Available tables: {tool_message.content}")
    return {"messages": [tool_call_message, tool_message, response]}

In [ ]:
def call_get_schema(state:MessagesState):
    llm_with_tools = llm.bind_tools([get_schema_tool], tool_choice="any")
    response = llm_with_tools.invoke(state["messages"])
    return {"messages":[response]}
    

In [ ]:
generate_query_system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
""".format(
    dialect=db.dialect,
    top_k=5,
)

In [ ]:
# Node to generate the SQL QUERY


def generate_query(state: MessagesState):
    system_message = {"role": "system", "content": generate_query_system_prompt}

    llm_with_tools = llm.bind_tools([run_query_tool])
    response = llm_with_tools.invoke([system_message] + state["messages"])

    return {"messages": [response]}

### Query Evaluator Agent

In [ ]:
validate_query_system_prompt = """
You are a SQL expert with a strong attention to detail.
Double check the {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes,
just reproduce the original query.

You will call the appropriate tool to execute the query after running this check.
""".format(dialect=db.dialect)

In [ ]:
def check_query(state: MessagesState):
    system_message = {
        "role": "system",
        "content": validate_query_system_prompt,
    }

    # Generate an artificial user message to check
    tool_call = state["messages"][-1].tool_calls[0]
    user_message = {"role": "user", "content": tool_call["args"]["query"]}
    llm_with_tools = llm.bind_tools([run_query_tool], tool_choice="any")
    response = llm_with_tools.invoke([system_message, user_message])
    response.id = state["messages"][-1].id

    return {"messages": [response]}

In [ ]:
def should_continue(state: MessagesState) -> Literal[END, "check_query"]:
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return END
    else:
        return "check_query"

In [ ]:
builder = StateGraph(MessagesState)
builder.add_node(list_tables)
builder.add_node(call_get_schema)
builder.add_node(get_schema_node, "get_schema")
builder.add_node(generate_query)
builder.add_node(check_query)
builder.add_node(run_query_node, "run_query")

builder.add_edge(START, "list_tables")
builder.add_edge("list_tables", "call_get_schema")
builder.add_edge("call_get_schema", "get_schema")
builder.add_edge("get_schema", "generate_query")
builder.add_conditional_edges(
    "generate_query",
    should_continue,
)
builder.add_edge("check_query", "run_query")
builder.add_edge("run_query", "generate_query")

agent = builder.compile()

In [ ]:
display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
def chat_with_agent(message, history):
    """
    Process user message and return agent's response
    """
    response = ""
    
    # Stream the agent's response
    for step in agent.stream(
        {"messages": [{"role": "user", "content": message}]},
        stream_mode="values",
    ):
        # Get the last message content
        last_message = step["messages"][-1]
        if hasattr(last_message, 'content'):
            response = last_message.content
    
    return response

In [ ]:
def display_tables():
    tool_call = {
        "name": "sql_db_list_tables",
        "args": {},
        "id": "abc123",
        "type": "tool_call",
    }
    tables = list_tables_tool(tool_call)
    return tables

In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("#  Humaanize DB - Query your SQL Database with Human Language")

    with gr.Row():
        # Left column: Chat interface
        with gr.Column(scale=3):
            gr.Markdown("Ask questions about music tracks, genres, and get insights!")

            chatbot = gr.Chatbot(
                height=500,
                bubble_full_width=False,
            )

            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Ask about genres, tracks, or statistics...",
                    show_label=False,
                    scale=4,
                )
                submit = gr.Button("Send", scale=1, variant="primary")

            with gr.Row():
                clear = gr.Button("Clear Chat")

            gr.Examples(
                examples=[
                    "Which genre on average has the longest tracks?",
                    "What are the top 5 most popular genres?",
                    "How many tracks are in each genre?",
                ],
                inputs=msg,
            )

        # Right column: Tables panel
        with gr.Column(scale=1):
            gr.Markdown("### 📊 Available Tables")
            tables_display = gr.Textbox(
                label="Database Tables",
                value=display_tables(),
                lines=10,
                interactive=False,
                max_lines=15,
            )
            refresh_btn = gr.Button("🔄 Refresh Tables", size="sm")
            refresh_btn.click(display_tables, outputs=tables_display)

    # Chat functionality
    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        bot_response = chat_with_agent(user_message, history)
        history[-1][1] = bot_response
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    submit.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()